# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [ ]:
    import pandas as pd
    columns = [
        'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
        'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
        'native-country', 'income'
    ]
    adult_dt = (pd.read_csv('/Users/Jakub_1/DSI/production/05_src/data/adult/adult.data', header = None, names = columns)
                .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [4]:
from sklearn.model_selection import train_test_split

X = adult_dt.drop(columns=['income'])
Y = adult_dt['income']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

print("Training set shape:", X_train.shape, Y_train.shape)
print("Testing set shape:", X_test.shape, Y_test.shape)


Training set shape: (22792, 14) (22792,)
Testing set shape: (9769, 14) (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

The random state (42) of the function is a random seed generator, by setting the seed we can ensure that the split can be reproduced (albeit it being random for this particular case) so that makes it useful for others to recreate our work. Had we not set a random state then it would be difficult for others to reproduce our work.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

numerical_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

num_transforms = Pipeline(steps=[
    ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('robust_scaler', RobustScaler())
])

cat_transforms = Pipeline(steps=[
    ('simple_imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot_encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num_transforms', num_transforms, numerical_cols),
    ('cat_transforms', cat_transforms, categorical_cols)
])

print(preprocessor)


ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('knn_imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('robust_scaler',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot_encoder',
                                                  OneHotEncoder(drop='first',
                                            

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

numerical_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

num_transforms = Pipeline(steps=[
    ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('robust_scaler', RobustScaler())
])

cat_transforms = Pipeline(steps=[
    ('simple_imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot_encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num_transforms', num_transforms, numerical_cols),
    ('cat_transforms', cat_transforms, categorical_cols)
])

model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])

print(model_pipeline)



Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('knn_imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('robust_scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                            

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [ ]:
from sklearn.model_selection import cross_validate

scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

cv_results = cross_validate(
    model_pipeline, X_train, Y_train,  
    cv=5,  
    scoring=scoring,  
    return_train_score=True  
)

print("Training Results:")
print("Negative Log Loss:", cv_results['train_neg_log_loss'])
print("ROC AUC:", cv_results['train_roc_auc'])
print("Accuracy:", cv_results['train_accuracy'])
print("Balanced Accuracy:", cv_results['train_balanced_accuracy'])

print("\nValidation Results:")
print("Negative Log Loss:", cv_results['test_neg_log_loss'])
print("ROC AUC:", cv_results['test_roc_auc'])
print("Accuracy:", cv_results['test_accuracy'])
print("Balanced Accuracy:", cv_results['test_balanced_accuracy'])


/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Training Results:
Negative Log Loss: [-0.08172696 -0.0812232  -0.0812362  -0.08149278 -0.08140081]
ROC AUC: [1. 1. 1. 1. 1.]
Accuracy: [1. 1. 1. 1. 1.]
Balanced Accuracy: [1. 1. 1. 1. 1.]

Validation Results:
Negative Log Loss: [-0.34963804 -0.35974188 -0.35011992 -0.34505114 -0.37187201]
ROC AUC: [0.90476282 0.90387673 0.90438349 0.90581487 0.90303887]
Accuracy: [0.85040579 0.84645756 0.85585783 0.8571742  0.85761299]
Balanced Accuracy: [0.77341767 0.76866029 0.77902132 0.77865939 0.7786413 ]


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [ ]:
import pandas as pd

cv_results_df = pd.DataFrame(cv_results)

cv_results_df_sorted = cv_results_df.sort_values(by='test_neg_log_loss', ascending=False)

cv_results_df_sorted.reset_index(drop=True, inplace=True)  
cv_results_df_sorted


,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
0,11.852358,0.315033,-0.345051,-0.081493,0.905815,1.0,0.857174,1.0,0.778659,1.0
1,10.732933,0.321330,-0.349638,-0.081727,0.904763,1.0,0.850406,1.0,0.773418,1.0
2,15.098829,0.331511,-0.350120,-0.081236,0.904383,1.0,0.855858,1.0,0.779021,1.0
3,12.810478,0.626016,-0.359742,-0.081223,0.903877,1.0,0.846458,1.0,0.768660,1.0
4,11.198138,0.248355,-0.371872,-0.081401,0.903039,1.0,0.857613,1.0,0.778641,1.0


Calculate the mean of each metric. 

In [ ]:
mean_metrics = cv_results_df.mean()


mean_metrics[['train_neg_log_loss', 'test_neg_log_loss', 'train_roc_auc', 'test_roc_auc', 
              'train_accuracy', 'test_accuracy', 'train_balanced_accuracy', 'test_balanced_accuracy']]


train_neg_log_loss        -0.081416
test_neg_log_loss         -0.355285
train_roc_auc              1.000000
test_roc_auc               0.904375
train_accuracy             1.000000
test_accuracy              0.853502
train_balanced_accuracy    1.000000
test_balanced_accuracy     0.775680
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [ ]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

model_pipeline.fit(X_train, Y_train)

Y_pred = model_pipeline.predict(X_test)            
Y_pred_proba = model_pipeline.predict_proba(X_test)  

test_metrics = {
    'neg_log_loss': log_loss(Y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y_test, Y_pred),
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)
}

test_metrics


{'neg_log_loss': 0.38453972527136493,
 'roc_auc': 0.9011762015480959,
 'accuracy': 0.8544375063977889,
 'balanced_accuracy': 0.7743956681605044}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

It's convenient because it recodes the data into binary which alligns with scikits requirements for binary classification metrics (and simplifies the interpretation of the results). It makes it easier to use metrics further down in teh assignment without having to convert when we get to them.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.